# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [57]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [16]:
path = os.path.join(os.getcwd(),"temperaturedata.csv")
data = pd.read_csv(path)
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,Kaitangata,-46.2817,169.8464,9.96,70,6,1.08,NZ,1624237789
1,Muros,42.7762,-9.0603,15.13,93,52,3.29,ES,1624237789
2,Hithadhoo,-0.6000,73.0833,28.64,70,100,5.40,MV,1624237789
3,Pisco,-13.7000,-76.2167,17.23,82,90,5.14,PE,1624237790
4,Bluff,-46.6000,168.3333,6.90,81,31,2.66,NZ,1624237790


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
#configure gmaps
gmaps.configure(api_key=g_key)
# store locations
locations = data[["Lat","Lng"]].astype(float)
# store weights
humidity_percent = data["Humidity"].astype(float)

In [13]:
# Create a humidity heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=humidity_percent, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
data2 = data
# dropping rows with clouds
data2 = data2.loc[(data2["Cloudiness"] == 0)]
# Dropping rows with strong winds
data2 = data2.loc[(data2["Windspeed"] < 10)]
# Dropping rows with very cold max temps
data2 = data2.loc[(data2["Max Temp"] < 50)]
data2 = data2.loc[(data2["Max Temp"] > 40)]
# displaying data
data2.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
214,Guadalupe Victoria,32.2869,-115.1050,42.75,23,0,5.56,MX,1624237862
484,Pahrump,36.2083,-115.9839,43.36,7,0,2.68,US,1624237948


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [59]:
hotel_df = data2
hotel_df["Hotel Name"] = ""
target_search = "Hotel"
target_type = "Lodging"
target_radius = 5000

params = {
    'keyword':target_search,
    'radius':target_radius,
    'type':target_type,
    'key':g_key
}

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    params["location"] = f"{latitude},{longitude}"

    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(endpoint, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))

    hotels = response["results"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels[0]["name"]
    except (KeyError, IndexError):
        print("No Hotels found")


In [60]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
214,Guadalupe Victoria,32.2869,-115.1050,42.75,23,0,5.56,MX,1624237862,Hotel Dial
484,Pahrump,36.2083,-115.9839,43.36,7,0,2.68,US,1624237948,"Holiday Inn Express & Suites Pahrump, an IHG H..."


In [67]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)

In [68]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)



# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))